In [1]:
# Install necessary libraries (Earth Engine)
!pip install earthengine-api

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Import the Earth Engine API
import ee

# Authenticate and initialize Earth Engine with the project ID
def authenticate_and_initialize_earth_engine(project_id):
    try:
        # Authenticate the user
        ee.Authenticate()  # This will prompt OAuth authentication

        # Initialize Earth Engine with the specified project ID
        ee.Initialize(project=project_id)
        print(f"Earth Engine initialized with project: {project_id}")
    except Exception as e:
        print(f"Error during authentication or initialization: {e}")

In [4]:
# Function to export Sentinel-2 image to Google Drive
def export_sentinel2_image_to_drive(latitude, longitude, start_date, end_date, image_name):
    # Define a point around which to get the image
    point = ee.Geometry.Point([longitude, latitude])

    # Load the Sentinel-2 image collection and filter by date and location
    collection = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
                  .filterBounds(point)
                  .filterDate(start_date, end_date)
                  .sort('CLOUD_COVER', True))  # Sort by cloud cover to get the least cloudy image

    # Get the first image in the filtered collection
    image = collection.first()

    # Select relevant bands (e.g., RGB bands: B4, B3, B2)
    selected_bands = image.select(['B4', 'B3', 'B2'])

    # Cast all bands to UInt16 to ensure data types are consistent
    #selected_bands = selected_bands.cast(ee.PixelType.uint16())

    # Define the region (smaller bounding box around the point, e.g., 1 km buffer)
    region = point.buffer(1000).bounds()

    # Create an export task to Google Drive
    task = ee.batch.Export.image.toDrive(
        image=selected_bands,
        description=image_name,
        folder='EarthEngineImages',  # Folder name in Google Drive
        fileNamePrefix=image_name,
        region=region.getInfo()['coordinates'],
        scale=30,  # Resolution in meters
        crs='EPSG:4326',  # Coordinate Reference System
        fileFormat='GeoTIFF'
    )

    # Start the export task
    task.start()
    print(f"Export task started to Google Drive for {image_name}.")

In [5]:
# Main code execution, Set up project details
project_id = 'genai-agent-hack-2024'  # Replace with your actual project ID

# Authenticate and initialize Earth Engine
authenticate_and_initialize_earth_engine(project_id)

# Example usage: export a Sentinel-2 image for a specific location and date
latitude = 37.7749   # Latitude for San Francisco
longitude = -122.4194  # Longitude for San Francisco
start_date = '2021-06-01'
end_date = '2021-06-30'
image_name = 'sentinel2_sf'

export_sentinel2_image_to_drive(latitude, longitude, start_date, end_date, image_name)

Earth Engine initialized with project: genai-agent-hack-2024
Export task started to Google Drive for sentinel2_sf.
